In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from src.plotting_tools.SysHist import make_hist, make_sys_hist, SysHist
from src.data_tools.get_data import get_data, delta_r_cut
from src.plotting_tools.Bins import Bins, bins, binning_type
from src.plotting_tools.cms_format import cms_format_fig, cms_style
from src.plotting_tools.colors import color_fader
from src.data_tools.StackPlotter import get_stack_plotter
from src.plotting_tools.latexAssets import HTLT, RelMET
from src.plotting_tools.utils import ratio_plot_template


In [ ]:
cms_style()

In [ ]:
era = 'Run 2'
fstring_era = era.replace(' ', '_')
from src.assets.output_dir import output_dir
output_dir

In [ ]:
sp6 = get_stack_plotter(output_dir, '2016', bins='none')
sp7 = get_stack_plotter(output_dir, '2017', bins='none')
sp8 = get_stack_plotter(output_dir, '2018', bins='none')

In [ ]:
sp = sp6
sp.plot_df = pd.concat([sp6.plot_df , sp7.plot_df, sp8.plot_df])
#sp.plot_df = pd.concat([sp.plot_df , sp8.plot_df])
sp.plot_df.era = 2000
sp.era=2000

In [ ]:
# add missing weights
keys = []
for i, row in sp.plot_df.iterrows():
    keys += list(row.sys.keys())
unique = np.unique(keys)
unique = list(filter( lambda x: "Weight" in x, unique))

for i, row in sp.plot_df.iterrows():
    keys = list(row.sys.keys())
    # don't do for data
    if len(keys) == 0: continue
    for w in unique:
        if w not in keys: row.sys[w] = [row.sys[keys[0]][0]*0,row.sys[keys[0]][0]]
    

In [ ]:
reg = 'SR1'
feature = 'HTLT'
sp.rebin =0
sp.x_range = (-np.inf, np.inf)
fig, ax = plt.subplots(figsize=(12,12))
x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
x = sp.draw_signals(ax, feature, reg, make_density=0)

ax.legend(ncol=2)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e7)
cms_format_fig("Run 2", ax, "\emph{Simulation}")
ax.set_ylabel("Events")
ax.set_xlabel(HTLT + " [GeV]")
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))

In [ ]:
reg = 'SR2'
feature = 'HTLT'
sp.scale = 1
sp.rebin =0
sp.x_range = (-np.inf, np.inf)
fig, ax = plt.subplots(figsize=(12,12))
x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
x = sp.draw_signals(ax, feature, reg, make_density=0)

ax.legend(ncol=2)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e7)
cms_format_fig("Run 2", ax, "\emph{Simulation}")
ax.set_ylabel("Events")
ax.set_xlabel(HTLT + " [GeV]")
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
sp.scale = 1

In [ ]:
reg = 'SR1'
feature = 'RelMET'
#scale by binning
sp.scale = 1
sp.rebin =0
sp.x_range = (-np.inf, np.inf)
fig, ax = plt.subplots(figsize=(12,12))
x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
x = sp.draw_signals(ax, feature, reg, make_density=0)
""
ax.legend(ncol=2)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e7)
cms_format_fig("Run 2", ax, "\emph{Simulation}")
ax.set_ylabel("Events")
ax.set_xlabel(RelMET)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
sp.scale = 1

In [ ]:
reg = 'SR2'
feature = 'RelMET'
#scale by binning
sp.scale = 1
sp.rebin =0
sp.x_range = (-np.inf, np.inf)
fig, ax = plt.subplots(figsize=(12,12))
x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
x = sp.draw_signals(ax, feature, reg, make_density=0)


ax.legend(ncol=2)
ax.set_yscale('log')
ax.set_ylim(1e-2, 1e7)
cms_format_fig("Run 2", ax, "\emph{Simulation}")
ax.set_ylabel("Events")
ax.set_xlabel(RelMET)
fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
sp.scale = 1

In [ ]:
def plot_reg_feature_ratio(reg, feature, xlabel='$\delta_{R}$'):
    fig, ax, rax = ratio_plot_template(figsize=(10,10))
    sp.scale = 1
    bhist = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
    _ = sp.draw_data(ax, feature, reg, make_density=0)
        
    #ratio
    x = bhist.calc_bin_centers()
    rax.plot(x, np.full(len(x), .5) , color='black', linestyle=':')
    rax.plot(x, np.full(len(x), 1) , color='black')
    rax.plot(x, np.full(len(x), 1.5) , color='black', linestyle=':') 
    rax.set_ylim(0,2)
    rax.set_ylabel('Obs./MC')
    if 'SR' in reg: 
        'none'
    else:
        _dhist = sp.make_data_hist(feature, reg).calc_ratio(bhist.nominal)
        rax.errorbar(_dhist.calc_bin_centers(), _dhist.nominal, yerr=_dhist.std, color='black',
                    label='Observed', ls='', marker='o')
    bhist.calc_ratio(bhist.nominal+1e-10).draw(rax, errorbar=False)
    
    
    ax.legend(ncol=2)
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 1e5)
    cms_format_fig("Run 2", ax, "\emph{Preliminary}")
    ax.set_ylabel("Events")
    ax.set_xlabel(xlabel)
    fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))

In [ ]:
reg = 'CR24'
feature = 'minGoodJetElDR'
plot_reg_feature_ratio(reg, feature, xlabel='$\Delta_{R}$ min(Electron, Jet)')

In [ ]:
reg = 'CR20'
feature = 'minGoodJetMuDR'
plot_reg_feature_ratio(reg, feature, xlabel='$\Delta_{R}$ min(Muon, Jet)')

In [ ]:
def make_uncertainty_dict(reg):
    nom, std = abcd[reg]['nom'], abcd[reg]['std']
    max_val, min_val, summed = np.max(std/nom), np.min(std/nom), np.sum(std)/np.sum(nom)
    avg = np.average(std/nom)
    return {"era": era, "reg": reg, 
            "max_val": max_val, "min_val": min_val, "summed": summed, "avg":avg}

In [ ]:
import pickle as pkl
firstHist=1
reg_dict = {}
uncertainty_list = []
for _era in ["2016", "2017", "2018"]:
    #if _era !="2016": continue
    outname="{}/abcd/abcd_dict_data_{}_ismc0.pkl".format(output_dir, _era)
    with open(outname,'rb') as f:
        abcd = pkl.load(f)
        for reg in abcd.keys():
            if'SR' in reg: uncertainty_list.append(make_uncertainty_dict(reg))
            
            if firstHist==1:
                reg_dict[reg] = SysHist.from_dict(abcd[reg])
            else:
                other = reg_dict[reg]
                self = SysHist.from_dict(abcd[reg])
                print(self.bin_edges, other.bin_edges)
                reg_dict[reg] += SysHist.from_dict(abcd[reg])
        firstHist=0
uncertainty_df = pd.DataFrame(uncertainty_list)

In [ ]:
uncertainty_df

In [ ]:
uncertainty_df.max(), uncertainty_df.min()

In [ ]:
def draw_dilep(reg, draw_sig=False, blinded=True):
    
    fig, ax, rax = ratio_plot_template(figsize=(10,10))
    
    
    feature = 'DiLepMass'

    sp.rebin = bins.bin_edges
    sp.x_range = (120, 400.5)
    bhist = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=1)
    #sp.scale = 10/1600
    dhist = sp.draw_data(ax, feature, reg, make_density=1, blinded=blinded)
    if 'SR' in reg and draw_sig: 
        _ = sp.draw_signals(ax, feature, reg, make_density=1, labels=False)
    
    if ('SR' in reg): kwargs = {"label": "ABCD", "zorder": 4}
    else: kwargs = {"label": "Observed Fit", "color": "red", "zorder": 4}
    reg_dict[reg].make_density_hist().draw(ax,**kwargs)
    
    ax.legend(ncol=2)
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 1e5)
    cms_format_fig(era, ax, "\emph{Preliminary}")
    ax.set_ylabel("Events per GeV")
    
    ax.set_xlabel('$m_{\ell\ell}$ [GeV]')
    
    #ratio
    x = bhist.calc_bin_centers()
    rax.plot(x, np.full(len(x), .5) , color='black', linestyle=':')
    rax.plot(x, np.full(len(x), 1) , color='black')
    rax.plot(x, np.full(len(x), 1.5) , color='black', linestyle=':') 
    rax.set_ylim(0,2)
    rax.set_ylabel('Obs./Fit')
    fhist = reg_dict[reg].rebin(dhist.bin_edges).make_density_hist()
    if 'SR' in reg and blinded: 
        rax.set_ylabel('Obs./ABCD')
    else:
        _dhist = dhist.calc_ratio(fhist.nominal) #sp.make_data_hist(feature, reg, blinded=blinded, make_density=1).calc_ratio(bhist.nominal)
        rax.errorbar(_dhist.calc_bin_centers(), _dhist.nominal, yerr=_dhist.std, color='black',
                    label='Observed', ls='', marker='o')
        
    fhist.calc_ratio(fhist.nominal).draw(rax, **kwargs)
    bhist.calc_ratio(fhist.nominal).draw(rax, zorder=1, errorbar=False, color='grey')
    if not draw_sig: 
        fig.savefig('{}/stack_plots/{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
    else:
        ax.legend(ncol=3)
        fig.savefig('{}/stack_plots/{}_{}_{}_signal.pdf'.format(output_dir, feature, fstring_era, reg))
    sp.scale = 1

In [ ]:
draw_dilep("SR1", blinded=False)
draw_dilep("SR1", blinded=False, draw_sig=True)

In [ ]:
draw_dilep("SR2", blinded=False)
draw_dilep("SR2", blinded=False, draw_sig=True)

In [ ]:
for reg in reg_dict.keys():
    print(reg)
    draw_dilep(reg, draw_sig=True)

In [ ]:
break

In [ ]:
#closure test
import pickle as pkl
firstHist=1
reg_dict = {}
uncertainty_list = []
for _era in ["2016", "2017", "2018"]:
    #if _era !="2016": continue
    outname="{}/abcd/abcd_dict_data_{}_ismc1.pkl".format(output_dir, _era)
    with open(outname,'rb') as f:
        abcd = pkl.load(f)
        for reg in abcd.keys():
            if'SR' in reg: uncertainty_list.append(make_uncertainty_dict(reg))
            
            if firstHist==1:
                reg_dict[reg] = SysHist.from_dict(abcd[reg])
            else:
                reg_dict[reg] += SysHist.from_dict(abcd[reg])
        firstHist=0
uncertainty_df = pd.DataFrame(uncertainty_list)

In [ ]:
def draw_dilep_closure(reg, draw_sig=False):
    
    fig, ax, rax = ratio_plot_template(figsize=(10,10))
    
    
    feature = 'DiLepMass'

    sp.rebin =reg_dict[reg].bin_edges
    sp.x_range = (120, 400)
    bhist = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=1)
    #sp.scale = 10/1600
    #_ = sp.draw_data(ax, feature, reg, make_density=0)
    if  draw_sig: _ = sp.draw_signals(ax, feature, reg, make_density=1)
    
    if ('SR' in reg): kwargs = {"label": "ABCD", "zorder": 4}
    else: kwargs = {"label": "Observed Fit", "color": "red", "zorder": 4}
    reg_dict[reg].make_density_hist().draw(ax,**kwargs)
    
    ax.legend(ncol=2)
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 1e5)
    cms_format_fig(era, ax, "\emph{Simulation}")
    ax.set_ylabel("Events per GeV")
    
    ax.set_xlabel('$m_{\ell\ell}$ [GeV]')
    
    #ratio
    x = bhist.calc_bin_centers()
    rax.plot(x, np.full(len(x), .5) , color='black', linestyle=':')
    rax.plot(x, np.full(len(x), 1) , color='black')
    rax.plot(x, np.full(len(x), 1.5) , color='black', linestyle=':') 
    rax.set_ylim(0,2)
    rax.set_ylabel('Obs./MC')
    if 'SR' in reg: 
        rax.set_ylabel('ABCD/MC')
        
    reg_dict[reg].make_density_hist().calc_ratio(bhist.nominal).draw(rax, **kwargs)
    bhist.calc_ratio(bhist.nominal).draw(rax, zorder=1, errorbar=False)
    if not draw_sig: 
        fig.savefig('{}/stack_plots/{}_{}_{}_closure.pdf'.format(output_dir, feature, fstring_era, reg))
    else:
        ax.legend(ncol=3)
        fig.savefig('{}/stack_plots/{}_{}_{}_signal_closure.pdf'.format(output_dir, feature, fstring_era, reg))
    sp.scale = 1

In [ ]:
for reg in reg_dict.keys():
    print(reg)
    draw_dilep_closure(reg, draw_sig=False)

In [ ]:
!ls -rt /eos/cms/store/group/phys_exotica/bffZprime/assets_june_23/data

In [ ]:
##
## Compare different dbs distributions
##
def compare_dbs(reg, feature, label):
    #scale by binning
    sp.scale = 1
    sp.rebin =0
    sp.x_range = (-np.inf, np.inf)
    fig, ax = plt.subplots(figsize=(12,12))
    #x = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=0)
    x = sp.draw_signals_compare_dbs(ax, feature, reg, make_density=1, 
                                    dbs_values = [0.04], mass_values = [125., 350.])
    x = sp.draw_signals_compare_dbs(ax, feature, reg, make_density=1, 
                                    dbs_values = [1.0], mass_values = [125., 350.],
                                   c1='#00da2b', c2='#e67e22')    
    
    ax.legend(ncol=2)
    ax.set_yscale('log')
    ax.set_ylim(1e-5, 1e0)
    cms_format_fig("Run 2", ax, "\emph{Simulation}")
    ax.set_ylabel("Events")
    ax.set_xlabel(label)
    fig.savefig('{}/stack_plots/dbs_comp_{}_{}_{}.pdf'.format(output_dir, feature, fstring_era, reg))
    sp.scale = 1

In [ ]:
reg = 'SR2'
feature = 'RelMET'
compare_dbs(reg, feature, RelMET)

In [ ]:
reg = 'SR1'
feature = 'RelMET'
compare_dbs(reg, feature, RelMET)

In [ ]:
reg = 'SR1'
feature = 'HTLT'
compare_dbs(reg, feature, HTLT)

In [ ]:
reg = 'SR2'
feature = 'HTLT'
compare_dbs(reg, feature, HTLT)

In [ ]:
#CRA-B test
from src.assets.regions import test_regions

In [ ]:
def draw_dilep_test_closure(reg, draw_sig=False):
    fig, ax, rax = ratio_plot_template(figsize=(10,10))
    sp = sp6
    
    feature = 'DiLepMass'
        
    sp.rebin = reg_dict['CR10'].bin_edges
    sp.x_range = (120, 400)
    bhist = sp.draw_background(ax, feature, reg, sys_label='Stat. + Sys.', errorbar=False, nom_color='grey', make_density=1)
    #sp.scale = 10/1600
    _ = sp.draw_data(ax, feature, reg, make_density=0)
    print(reg, _.calc_sum())
    if  draw_sig: _ = sp.draw_signals(ax, feature, reg, make_density=1)
    
    if ('SR' in reg): kwargs = {"label": "ABCD", "zorder": 4}
    else: kwargs = {"label": "Observed Fit", "color": "red", "zorder": 4}
    #reg_dict[reg].make_density_hist().draw(ax,**kwargs)
    
    ax.legend(ncol=2, title=reg.replace("_", " "))
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 1e5)
    cms_format_fig(era, ax, "\emph{Simulation}")
    ax.set_ylabel("Events per GeV")
    
    ax.set_xlabel('$m_{\ell\ell}$ [GeV]')
    
    #ratio
    x = bhist.calc_bin_centers()
    rax.plot(x, np.full(len(x), .5) , color='black', linestyle=':')
    rax.plot(x, np.full(len(x), 1) , color='black')
    rax.plot(x, np.full(len(x), 1.5) , color='black', linestyle=':') 
    rax.set_ylim(0,2)
    rax.set_ylabel('Obs./MC')
    if 'SR' in reg: 
        rax.set_ylabel('ABCD/MC')
        
    #reg_dict[reg].make_density_hist().calc_ratio(bhist.nominal).draw(rax, **kwargs)
    bhist.calc_ratio(bhist.nominal).draw(rax, zorder=1, errorbar=False)
    if not draw_sig: 
        fig.savefig('{}/stack_plots/{}_{}_{}_closure.pdf'.format(output_dir, feature, fstring_era, reg))
    else:
        ax.legend(ncol=3)
        fig.savefig('{}/stack_plots/{}_{}_{}_signal_closure.pdf'.format(output_dir, feature, fstring_era, reg))
    sp.scale = 1

In [ ]:
for reg in test_regions:
    print(reg)
    draw_dilep_test_closure(reg, draw_sig=False)